<div style="width: 50%; margin: 0 auto;">

# Maze Solving Workshop:
### *Presented by the WVU University Rover Challenge Team*
*By: Nathan Adkins*

<br>

# Outline:

### Part 0: Defining the Maze and the Robot 
### Part 1: Learning Robot Operation
### Part 2: First Algorithmic Maze Solver
### Part 3: Non-Continuous Maze Solver
### Part 4: Try at Home: Dijkstra's Shortest Path

<div style="max-width: 33%">

# Part 0: Defining the Maze and the Robot 

In the code below, the *Maze* class defines the maze the robot will be traversing. The maze is constituted by a 2 dimensional grid of cells. Each cell has a state indicating a quality about the cell. The maze has 2 primary states with 4 other states. The states indicate information about individual cells in the maze.

Here are the states of the maze:
- *UNPOPULATED*
- *POPULATED*
- *GOAL*
- *ROBOT*
- *PATH*
- *ROBOT_AT_GOAL*

Now that we have the states of the maze defined, we can generate mazes using the *UNPOPULATED* and *POPULATED* states. These states will describe the travserable and non-travserable portions of the maze. For each of the activities in this notebook, algorithms will be tested on pre-made mazes. Optionally, to be able to test the robustness of the maze solving algorithms, a [recursive backtracking](https://en.wikipedia.org/wiki/Backtracking) depth first search algorithm has been implemented to generate random mazes. 

Ironically, similar mathematical techniques can be leveraged to solve mazes. Recursive backtracking is commonly used in game theory, [combinatorial optimization](https://en.wikipedia.org/wiki/Combinatorial_optimization), and in solving Sudoku.
Note:<br>
Normal Python code does not look like this. The code block below has been condensed to save space in this notebook. However, the more visually organized code can be found in organized_maze_code.py.

</div>

In [ ]:
import random, time, math
import numpy as np 
import tkinter as tk
from typing import Callable
from os import system, name

def clear_terminal_output():
    if name == 'nt': system('cls')
    else: system('clear')
    
class Maze:
    UNPOPULATED = 0
    POPULATED = 1
    GOAL = 3
    ROBOT = 4 
    PATH = 5
    ROBOT_AT_GOAL = 6
    COLORS = {
        UNPOPULATED: '#ffffff',
        POPULATED: '#757575',
        GOAL: '#cc0000',
        ROBOT: '#6a7b8b',
        PATH: '#c0d7ec',
        ROBOT_AT_GOAL: '#9B3E46',
    }
    DEFAULT_WINDOW_WIDTH_PX = 1000
    def __init__(self, size: int, algorithm_to_run: Callable, gui_title: str):
        clear_terminal_output()
        def algorithm_runner_wrapper(algorithm_to_run):
            self.run_algorithm_button.config(state=tk.DISABLED)
            try: algorithm_to_run()
            except Exception as exception: print(exception); print("To restart, close the maze window")
        def _depth_first_search_maze(x, y) -> None:
            directions = [(0, 2), (2, 0), (0, -2), (-2, 0)]
            random.shuffle(directions)
            for dx, dy in directions:
                nx, ny = x + dx, y + dy
                if 0 <= nx < self.size and 0 <= ny < self.size and self.maze_state[nx, ny] == self.POPULATED:
                    self.maze_state[x + dx // 2][y + dy // 2] = self.UNPOPULATED
                    self.maze_state[nx, ny] = self.UNPOPULATED
                    _depth_first_search_maze(nx, ny)
        self.size = size
        self.rectangles = {}
        self.maze_state = np.full((size, size), self.POPULATED)
        _depth_first_search_maze(0,0)
        self.DEFAULT_PIXELS_PER_MAZE_CELL = math.ceil(self.DEFAULT_WINDOW_WIDTH_PX / self.size)
        self.maze_gui = tk.Tk(); self.maze_gui.title(gui_title)
        self.frame = tk.Frame(self.maze_gui); self.frame.pack(pady=10, padx=10)
        self.run_algorithm_button = tk.Button(self.maze_gui, text="Run Algorithm", command= lambda: algorithm_runner_wrapper(algorithm_to_run)); self.run_algorithm_button.config(state=tk.DISABLED); self.run_algorithm_button.pack()
        self.canvas = tk.Canvas(self.frame, width=self.size * self.DEFAULT_PIXELS_PER_MAZE_CELL, height=self.size * self.DEFAULT_PIXELS_PER_MAZE_CELL); self.canvas.pack()
        self.speed_slider = tk.Scale(self.frame, from_=1, to=100, orient=tk.HORIZONTAL, label="Speed multiplier"); self.speed_slider.set(1); self.speed_slider.pack(fill=tk.X)
        self.update_gui()
    def update_gui(self) -> None:
        unit_size = self.DEFAULT_PIXELS_PER_MAZE_CELL
        for row in range(self.size):
            for col in range(self.size):
                x1, y1, x2, y2 = col * unit_size, row * unit_size, col * unit_size + unit_size, row * unit_size + unit_size
                cell_value = self.maze_state[row, col]
                if (row, col) not in self.rectangles: self.rectangles[(row, col)] = self.canvas.create_rectangle(x1, y1, x2, y2, fill=self.COLORS[cell_value], outline="black")
                else: self.canvas.itemconfig(self.rectangles[(row, col)], fill=self.COLORS[cell_value])
        self.maze_gui.update()
    def _make_empty(self):
        self.maze_state = np.zeros((self.size,self.size))
class Robot():
    DEFAULT_LOAD_DELAY_SEC = 0.2
    DEFAULT_MOVEMENT_TIME_STEP_SEC = 0.1
    def calc_position(position, movement): return [tuple(map(sum, zip(position, movement)))]
    def _is_valid_position(self, proposed_position: tuple[int]) -> bool: return all([0 <= coord < self.maze.size for coord in proposed_position]) and (len(proposed_position) == 2)
    def _is_free_space(self, proposed_position: tuple[int]) -> bool: return (self.maze.maze_state[proposed_position] != self.maze.POPULATED)

    def is_valid_movement(self, movement_delta) -> bool:
        proposed_new_position = tuple(map(sum, zip(self.current_position, movement_delta))) 
        return self._is_free_space(proposed_new_position) and self._is_valid_position(proposed_new_position)
    
    def _update_maze_gui(self, updated_position: tuple[int], updated_maze_state):
        if self._is_valid_position(updated_position): self.maze.maze_state[updated_position] = updated_maze_state; self.maze.update_gui()
        else: raise ValueError(f"The value of the updated maze position {updated_position} is not within the maze")
    def __init__(self, maze: Maze, start_position: tuple[int], goal_position: tuple[int]):
        def _load_delay(): time.sleep(self.DEFAULT_LOAD_DELAY_SEC)
        self.maze = maze
        self.current_position = start_position
        self.right_from_robot = self.maze.maze_state[tuple(map(sum, zip(self.current_position, (0,1))))]
        self.left_from_robot = self.maze.maze_state[tuple(map(sum, zip(self.current_position, (0,-1))))]
        self.down_from_robot = self.maze.maze_state[tuple(map(sum, zip(self.current_position, (1,0))))]
        self.up_from_robot = self.maze.maze_state[tuple(map(sum, zip(self.current_position, (-1,0))))]
        try:
            self.at_goal = False

            self.maze_down_state = self.maze_up_state = self.maze.maze_state[tuple(map(sum, zip(self.current_position, (-1,0))))]
            if self._is_valid_position(start_position): self.start_position = start_position
            else: raise ValueError(f"The start position of the robot {start_position} is not within it's maze")
            if self._is_valid_position(goal_position): self.goal_position = goal_position
            else: raise ValueError(f"The goal position of the robot {goal_position} is not within it's maze")
            
            _load_delay(); self._update_maze_gui(self.goal_position, self.maze.GOAL)
            _load_delay(); self._update_maze_gui(self.current_position, self.maze.ROBOT)
            self.maze.run_algorithm_button.config(state=tk.NORMAL)
        except tk.TclError: print("There was an error with the maze window. Please run the code again")
    def _move(self, movement_delta: tuple[int]) -> bool:
        proposed_new_position = tuple(map(sum, zip(self.current_position, movement_delta)))
        if self.is_valid_movement(proposed_new_position): 
            old_position = self.current_position; self.current_position = proposed_new_position
            time.sleep( self.DEFAULT_MOVEMENT_TIME_STEP_SEC /(self.maze.speed_slider.get()))
            if old_position == self.start_position: self._update_maze_gui(old_position, self.maze.PATH)
            else: self._update_maze_gui(old_position, self.maze.PATH)
            if self.current_position == self.goal_position: self.at_goal = True; self._update_maze_gui(self.current_position,self.maze.ROBOT_AT_GOAL); print("\nThe robot has reached the goal\n")
            else: self._update_maze_gui(self.current_position,self.maze.ROBOT)
        else: movement_delta_dict = { (0,1) : 'right', (0,-1) : 'left', (-1,0) : 'up', (1,0) : 'down',}; raise ValueError(f"The robot tried moving {movement_delta_dict[movement_delta]}. This movement is not valid given the robot's current position")
    def move_right(self): self._move((0,1))
    def move_left(self): self._move((0,-1))
    def move_down(self): self._move((1,0))
    def move_up(self): self._move((-1,0))

: 

<div style="max-width: 33%">

# Part 1: Learning Robot Operation

Now that the Maze and Robot are defined, the Robot can be used to travserse the maze.

The maze_state contains values corresponding to one of the states mentioned in part 0 for each of the cells in the maze and are able to be accessed.

The following methods can be used to control the robot in the maze.
- *move_up*
- *move_down*
- *move_left*
- *move_right*

The text output below the code block will display error messages. Additionally, the Part 0 code before testing code or algorithms in subsequent parts.

</div>

In [2]:
def main():

    def part1_example_algorithm():
        while not robot.at_goal:
            robot.move_down()
            robot.move_right()        
            
    part1_maze_size = 11
    maze = Maze(part1_maze_size, part1_example_algorithm, "Part 1")
    maze._make_empty()
    robot = Robot(maze,(0,0),(part1_maze_size - 1, part1_maze_size - 1))
    maze.maze_gui.mainloop()

if __name__ == '__main__':
    main()

index 11 is out of bounds for axis 0 with size 11
To restart, close the maze window


# Part 2: First Algorithmic Maze Solver

<div style="max-width: 33%">

As you probably noticed in the previous part, handwriting code is annoying, time consuming, and does allow the robot to adapt. Ideally, when writing algorithms for path planning, you want to minimize the number of portions of code that make calls to control the robot. The general idea is to have a check on conditions about the environment that dictate where the robot goes. 

In this next part, we will be leveraging this idea to create an algorithm to solve the maze.

</div>

In [3]:
def main():

    def part2_algorithm():

        right = (0,1)
        left = (0,-1)
        down = (1,0)
        up = (-1,0)

        if robot.is_valid_movement(right):
            robot.move_right()
            last_movement = right

        elif robot.is_valid_movement(left):
            robot.move_left()
            last_movement = left

        elif robot.is_valid_movement(down):
            robot.move_down()
            last_movement = down

        elif robot.is_valid_movement(up):
            robot.move_up()
            last_movement = up


        while robot.is_valid_movement(last_movement):
            robot.move_down()
            pass

        

    maze = Maze(11, part2_algorithm, "Part 2")
    maze.maze_state = np.array([ 
        [0,1,0,0,0,1,0,0,0,0,0],
        [0,1,0,1,0,1,0,0,1,1,1],
        [0,1,0,1,0,1,1,0,1,0,0],
        [0,1,0,1,0,0,0,0,1,1,0],
        [0,1,0,1,1,1,1,0,0,0,0],
        [0,1,0,0,0,0,1,0,1,1,0],
        [0,1,1,1,1,0,1,0,1,0,0],
        [0,0,0,0,0,0,1,0,1,0,1],
        [0,1,1,1,1,0,1,0,1,0,1],
        [0,1,0,0,0,0,1,0,1,0,1],
        [0,1,0,1,1,1,1,0,1,0,0],])
       
    robot = Robot(maze,(0,0),(10,10))
    maze.maze_gui.mainloop()

if __name__ == '__main__':
    main()

index 11 is out of bounds for axis 0 with size 11
To restart, close the maze window


# Part 3: Non-Continuous Maze Solver

<div style="max-width: 33%">

</div>

In [4]:
def main():

    def testing():
        while not robot.at_goal:
            robot.move_down()
            robot.move_right()

    maze = Maze(51, testing, "Part 3")
    robot = Robot(maze,(0,0),(50,50))
    maze.maze_gui.mainloop()

if __name__ == '__main__':
    main()

The robot tried moving right. This movement is not valid given the robot's current position
To restart, close the maze window


# Part 4: Try at Home: Dijkstra's Shortest Path

<div style="max-width: 33%">

</div>

In [5]:
def main():

    def testing():
        while not robot.at_goal:
            robot.move_down()
            robot.move_right()

    maze = Maze(51, testing, "Part 4")
    robot = Robot(maze,(0,0),(50,50))
    maze.maze_gui.mainloop()

if __name__ == '__main__':
    main()